In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import shutil
import string
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords #Word Stop
from nltk.tokenize import word_tokenize #Tokenization & Word Stop

In [23]:
to_train = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_dataset_notpreprocessed.csv')
to_train['Combined_Text'] = to_train['Title'] + ' ' +  to_train['Description'] + ' ' + to_train['Component']
data = to_train.copy()
to_train.head()

,Issue_id,Priority,Title,Description,Component,Combined_Text
0,1,P3,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resour...,Team,Usability issue with external editors (1GE6IRL...
1,13,P3,CC Discussion: local versioning (1GAT3PL),Would it make sense to have a (the?) projects ...,Team,CC Discussion: local versioning (1GAT3PL) Woul...
2,12,P3,Manage/unmanage support and policies (1GALAEG),Currently every new resource added to a shared...,Team,Manage/unmanage support and policies (1GALAEG)...
3,11,P3,API: ISharingManager::load mapping vcm project...,Jean-Michel (08/02/2001 1:38:48 PM); ; The ISh...,Team,API: ISharingManager::load mapping vcm project...
4,10,P3,API - VCM event notification (1G8G6RR),There seems to be a need for a VCM Event notif...,Team,API - VCM event notification (1G8G6RR) There s...


In [24]:
to_get_topics = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_dataset.csv')
to_train['Topic'] = to_get_topics['Topic']


In [25]:
topic_zero_data = to_train[to_train['Topic']==0]

In [26]:

extract_dir = '/home/rpierson/PiersonREU/extracted'
topic_zero_path = os.path.join(extract_dir, 'topic_zero_data.csv')
topic_zero_data.to_csv(topic_zero_path, index = False)
topic_zero_data

,Issue_id,Priority,Title,Description,Component,Combined_Text,Topic
177,146,P3,UnknownHostException: Repositories (1GEWRWG),UnknownHostException is not caught when the ho...,Team,UnknownHostException: Repositories (1GEWRWG) U...,0
194,234,P3,NPE in sync (1GJ4W10),Did a sync of Eclipse UI. It came up in the c...,Team,NPE in sync (1GJ4W10) Did a sync of Eclipse UI...,0
246,217,P3,UI BUG TO BE FILED (1GHQOGF),While minimizing an Eclipse running in Eclipse...,Team,UI BUG TO BE FILED (1GHQOGF) While minimizing ...,0
337,1749,P1,Debugger did not show source for type; and gen...,There are two problems here:; problem 1:; - ra...,UI,Debugger did not show source for type; and gen...,0
348,1811,P3,Scripted Handler Example does not run (1GENWSM),scripted handler example does not run in 116; ...,Scripting,Scripted Handler Example does not run (1GENWSM...,0
...,...,...,...,...,...,...,...
68028,229022,P3,[Progress] Display#getShells returns disposed ...,I20080427-2000; ; To reproduce the latest I-Bu...,UI,[Progress] Display#getShells returns disposed ...,0
68032,229097,P3,NPE when using OpenCheatSheetFromHelpAction,Created attachment 97790; Demo project; ; Buil...,User Assistance,NPE when using OpenCheatSheetFromHelpAction Cr...,0
68079,229473,P3,[Contributions] ShowInMenu dispose is attempti...,I20080429-0100; ; I got this by quitting Eclip...,UI,[Contributions] ShowInMenu dispose is attempti...,0
68108,229712,P3,NPE in MenuServiceFactory likely from File->Re...,I found two of these in my .log. I see restar...,UI,NPE in MenuServiceFactory likely from File->Re...,0


In [27]:
n = len(topic_zero_data)
split_index = int(.8 * n)
train_dataset = topic_zero_data.iloc[:split_index]
validation_dataset = topic_zero_data.drop(train_dataset.index)
validation_dataset

,Issue_id,Priority,Title,Description,Component,Combined_Text,Topic
55402,146750,P3,[DataBinding][CompositeTable] Crash when dragg...,Run CompositeTableTest; Drag scroll bar down; ...,UI,[DataBinding][CompositeTable] Crash when dragg...,0
55404,146770,P3,SWTError: No more handles in Java support for ...,OS: Windows20003server; eclipse.buildId=I20060...,SWT,SWTError: No more handles in Java support for ...,0
55540,147994,P3,[WorkbenchParts] Graphic is disposed,Hello;; ; I have written two RCP applications ...,UI,[WorkbenchParts] Graphic is disposed Hello;; ;...,0
55542,148025,P2,[Viewers] TreeViewer has empty entries when re...,3.3 pre M1; ; When you select all of the entri...,UI,[Viewers] TreeViewer has empty entries when re...,0
55573,148270,P3,[Forms] WID when Form disposed while busy,3.2 RC5; ; An editor using a Form control got ...,User Assistance,[Forms] WID when Form disposed while busy 3.2 ...,0
...,...,...,...,...,...,...,...
68028,229022,P3,[Progress] Display#getShells returns disposed ...,I20080427-2000; ; To reproduce the latest I-Bu...,UI,[Progress] Display#getShells returns disposed ...,0
68032,229097,P3,NPE when using OpenCheatSheetFromHelpAction,Created attachment 97790; Demo project; ; Buil...,User Assistance,NPE when using OpenCheatSheetFromHelpAction Cr...,0
68079,229473,P3,[Contributions] ShowInMenu dispose is attempti...,I20080429-0100; ; I got this by quitting Eclip...,UI,[Contributions] ShowInMenu dispose is attempti...,0
68108,229712,P3,NPE in MenuServiceFactory likely from File->Re...,I found two of these in my .log. I see restar...,UI,NPE in MenuServiceFactory likely from File->Re...,0


In [28]:
topic_zero_train_priority = train_dataset['Priority']
topic_zero_train_data = train_dataset['Combined_Text']
topic_zero_validation_priority = validation_dataset['Priority']
topic_zero_validation_data = validation_dataset['Combined_Text']
def extract_description(topic_zero_train_data):
    # Using regular expression to extract text after the first parentheses
    description_train = re.sub(r'^\([^)]+\)\s*', '', topic_zero_train_data)
    return description_train.strip()
def extract_description(topic_zero_validation_data):
    # Using regular expression to extract text after the first parentheses
    description_validate = re.sub(r'^\([^)]+\)\s*', '', topic_zero_validation_data)
    return description_validate.strip()

In [29]:
print(topic_zero_train_data.shape)
print(topic_zero_validation_data.shape)

(2265,)
(567,)


In [30]:
vectorizer = CountVectorizer(stop_words = 'english', tokenizer = word_tokenize, lowercase = True)
topic_zero_train_data = vectorizer.fit_transform(topic_zero_train_data)
topic_zero_validation_data = vectorizer.transform(topic_zero_validation_data)

/home/rpierson/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [31]:
gnb = GaussianNB()
y_pred = gnb.fit(topic_zero_train_data.toarray(), topic_zero_train_priority).predict(topic_zero_validation_data.toarray())

In [32]:
print("Number of mislabeled points out of a total %d points : %d"
      % (topic_zero_validation_data.shape[0], (topic_zero_validation_priority != y_pred).sum()))

Number of mislabeled points out of a total 567 points : 51


In [33]:
priority_dispersion = topic_zero_data.groupby('Priority').count()

In [34]:
priority_dispersion

,Issue_id,Title,Description,Component,Combined_Text,Topic
Priority,,,,,,
P1,261,261,261,261,261,261
P2,323,323,323,323,323,323
P3,2208,2208,2208,2208,2208,2208
P4,22,22,22,22,22,22
P5,18,18,18,18,18,18


In [35]:
mismatches = (topic_zero_validation_priority != y_pred).groupby(topic_zero_validation_priority['Priority'])

KeyError: 'Priority'